Datos dependientes del padrón:

Padrón = 110197

Lambda = 1000

Sigma = 97/100+0.1= 1.07

b = 1.5

# Consigna

La empresa “El comandante Fort S.A” debe planificar aproximadamente sus estimaciones de producción para el próximo año. Se trabaja con 2 productos. El producto A es de producción barata y precio de venta actual de $4000 por unidad, mientras que el producto B es de un mayor costo y calidad con un precio de venta actual de $14000.

La empresa quiere obtener algunos datos estimados del año que viene sabiendo que, puede producir,mensualmente, 4000 unidades de A o 1500 unidades de B o una combinación de ambos. Al mismo tiempo, cada mes, se debería satisfacer una demanda mínima para cada producto. En el caso del producto A, la demanda mínima mensual se distribuye como una Poisson(1000), mientras que en el producto B, la demanda mínima mensual se distribuye como una binomial(p=0,85,n=300), dónde n representa a clientes “fieles” al producto, según registros históricos, y p la probabilidad de que adquieran el producto ese mes.

Al mismo tiempo, al finalizar cada mes, el departamento de compras estima que se darán aumentos mensuales en los precios de ventas para cada producto el primer día de cada mes (previo a comenzar a vender). Para el producto A, los aumentos mensuales se distribuyen normalmente(u=2,sigma=1.07) en términos porcentuales, mientras que en el producto B serán del tipo uniforme(a=0,b=1.5), también en términos porcentuales

- Año tiene 12 meses
- Los aumentos se dan sobre el precio del producto en el momento, NO sobre el precio inicial
- El primer mes se realiza con el precio actual

1) Se solicita simular el año e indicar:
	- El ingreso anual esperado en promedio
	- Los límites de confianza con un 80% del aumento anual en el producto B, representado porcentualmente
	
2) Generar 1000 valores simulados de:
	- El aumento anual del precio del producto A (en $)
	- La demanda mínima anual de ambos productos


In [1]:
import numpy as np
import pandas as pd
import pulp as pl
from scipy import stats

In [ ]:
# Defino mis configuraciones tanto de la simulación 
# como de parametros iniciales

# Semilla para reproducibilidad
np.random.seed(42) # Referencia a Asimov

# Parámetros iniciales
precio_inicial_A = 4000  # Precio inicial del producto A
precio_inicial_B = 14000  # Precio inicial del producto B
max_unidades_A = 4000  # Capacidad máxima mensual de producción de A
max_unidades_B = 1500  # Capacidad máxima mensual de producción de B
meses = 12  # Cantidad de meses a simular
num_simulaciones = 1000  # Número de simulaciones. Uso 1000 para no tener que replantear todo para el c y el d


# Acá empiezo a armar las funciones que voy a usar para la simulación.
Prefiero dejarlo como funciones para que sea más fácil de leer y entender (y que sea todo por separado así no me pierdo mucho si hay que corregir algo).

In [3]:
# Función para generar demanda mínima mensual en base a las consignas
def generar_demanda_minima():
    demanda_min_A = np.random.poisson(lam=1000)  # Poisson(lambda=1000)
    demanda_min_B = np.random.binomial(n=300, p=0.85)  # Binomial(n=300, p=0.85)
    return demanda_min_A, demanda_min_B


In [4]:
# Función para calcular aumentos de precios usando las distribuciones que nos dieron
#! b y sigma salen del padrón
# Esta función la voy a tener que llamar cada mes para calcular el aumento de precios compuesto 
def calcular_aumentos():
    aumento_A = np.random.normal(loc=2, scale=1.07) / 100  # Normal(u=2, sigma=1.07) en porcentaje
    aumento_B = np.random.uniform(low=0, high=1.5) / 100  # Uniforme(a=0, b=1.5) en porcentaje
    return aumento_A, aumento_B

In [ ]:
# Función para optimizar la producción mensual usando PuLP
def optimizar_produccion(precio_A, precio_B, demanda_min_A, demanda_min_B):
    # Creo el problema de maximización
    problema = pl.LpProblem("Maximizar_Ingresos", pl.LpMaximize)
    
    # Armo mis variables de decisión asumiendo que es imposible realizar ventas fraccionales
    A = pl.LpVariable("Unidades_A", lowBound=demanda_min_A, cat="Integer")
    B = pl.LpVariable("Unidades_B", lowBound=demanda_min_B, cat="Integer")
    
    # Función objetivo: maximizar ingresos
    problema += precio_A * A + precio_B * B, "Ingresos_Totales"
    
    # Restricción de capacidad de producción (reformulada para evitar dividir LpVariable por int)
    problema += A * 1500 + B * 4000 <= 4000 * 1500, "Restricción_Capacidad"
    
    # Resolver el problema
    problema.solve(pl.PULP_CBC_CMD(msg=False))
    
    # Obtener las cantidades óptimas
    unidades_A = pl.value(A)
    unidades_B = pl.value(B)
    
    # Calcular ingresos
    ingresos = pl.value(problema.objective)
    
    return unidades_A, unidades_B, ingresos



# Acá tengo que plantear las simulaciones

In [ ]:
# Realizar simulaciones
resultados = []
aumentos_acumulados_B = []
aumentos_acumulados_A_pesos = []  # Lista para guardar aumentos anuales de A en pesos
demandas_minimas_A_anuales = []   # Lista para guardar demandas mínimas anuales de A
demandas_minimas_B_anuales = []   # Lista para guardar demandas mínimas anuales de B


for _ in range(num_simulaciones):
    precio_A = precio_inicial_A
    precio_B = precio_inicial_B
    ingresos_anuales = 0
    # Creo acá unas variables para hacer el aumento compuesto
    aumento_acumulado_B = 1  # Factor multiplicativo para el producto B
    aumento_acumulado_A = 1  # Factor multiplicativo para el producto A
    
    # Inicializo las demandas mínimas anuales
    demanda_min_anual_A = 0
    demanda_min_anual_B = 0
    
    for mes in range(meses):
        # Meto condición para que el primer mes no tenga aumento
        if mes > 0:
            aumento_A, aumento_B = calcular_aumentos()
            precio_A *= (1 + aumento_A)
            precio_B *= (1 + aumento_B)
            aumento_acumulado_B *= (1 + aumento_B)
            aumento_acumulado_A *= (1 + aumento_A)
        
        # Generar demanda mínima
        demanda_min_A, demanda_min_B = generar_demanda_minima()
        
        # Acumular demanda mínima anual
        demanda_min_anual_A += demanda_min_A
        demanda_min_anual_B += demanda_min_B
        
        # Optimizar producción
        unidades_A, unidades_B, ingresos_mes = optimizar_produccion(
            precio_A, precio_B, demanda_min_A, demanda_min_B
        )
        
        # Acumular ingresos mensuales
        ingresos_anuales += ingresos_mes
    
    # Guardar resultados de la simulación
    resultados.append(ingresos_anuales)
    aumentos_acumulados_B.append((aumento_acumulado_B - 1) * 100)  # Convertir a porcentaje
    
    # Guardar el aumento anual del precio de A en pesos (precio final - precio inicial)
    aumento_A_pesos = precio_A - precio_inicial_A
    aumentos_acumulados_A_pesos.append(aumento_A_pesos)
    
    # Guardar las demandas mínimas anuales
    demandas_minimas_A_anuales.append(demanda_min_anual_A)
    demandas_minimas_B_anuales.append(demanda_min_anual_B)


# Calcular estadísticas
ingreso_anual_promedio = np.mean(resultados)
percentil_10_B = np.percentile(aumentos_acumulados_B, 10)
percentil_90_B = np.percentile(aumentos_acumulados_B, 90)




# Acá paso todo a CSV usando Pandas (que es lo que acostumbro a usar)

In [ ]:
#Armo mis dataframes para los csv

df_resultados = pd.DataFrame({
    'ing_an': [ingreso_anual_promedio],
    'lim_inf_b': [percentil_10_B],
    'lim_sup_b': [percentil_90_B]
})

df_aumentos = pd.DataFrame({
    'aum_a': aumentos_acumulados_A_pesos
})

df_demandas_minimas = pd.DataFrame({
    'dem_min_a': demandas_minimas_A_anuales,
    'dem_min_b': demandas_minimas_B_anuales
})

# Guardar resultados en CSV
df_resultados.to_csv('informe_110197.csv', index=False)
df_aumentos.to_csv('aumentos_110197.csv', index=False)
df_demandas_minimas.to_csv('demandas_minima_110197.csv', index=False)
